In [11]:
# Load Dependencies:
from IrisBackendv3.codec.settings import set_codec_standards
from IrisBackendv3.data_standards.prebuilt import add_to_standards, ALL_PREBUILT_MODULES
from IrisBackendv3.data_standards.logging import logger as DsLogger
from IrisBackendv3.codec.magic import Magic
from IrisBackendv3.codec.payload import WatchdogCommandPayload, CommandPayload
from IrisBackendv3.codec.payload_collection import EnhancedPayloadCollection
from IrisBackendv3.codec.packet import Packet, IrisCommonPacket, parse_packet
from IrisBackendv3.codec.metadata import DataPathway, DataSource
from IrisBackendv3.data_standards import DataStandards
import scapy.all as scp
from termcolor import cprint

from __command_aliases import get_command

# Re-build the `DataStandards` fresh from latest fprime and add in all prebuilt
# (WD Telem) Modules.
# These `DataStandards` will serve as TMTC definitions be used by the `Codec`
# layer used by the `Transceiver` layer to interpret packets.
# TODO: Rebuild this once in the main IPC module, cache it, and load from cache in other IPC apps.
DsLogger.setLevel('CRITICAL')
standards = DataStandards.build_standards()
add_to_standards(standards, ALL_PREBUILT_MODULES)
set_codec_standards(standards)

In [12]:
# Build Command Bytes:
cmd, param = 'ReportStatus', True

print(cmd, '  :  ', param)

# Build command(s):
pathway, magic, command_name, kwargs, telem_pathway = get_command(cmd, param)
print(pathway, magic, command_name, kwargs)

command_payload_type = {
    Magic.WATCHDOG_COMMAND: WatchdogCommandPayload,
    Magic.RADIO_COMMAND: CommandPayload,
    Magic.COMMAND: CommandPayload
}[magic]

module, command = standards.global_command_lookup(command_name)

payloads = EnhancedPayloadCollection(
    CommandPayload=[
        command_payload_type(
            pathway=pathway,
            source=DataSource.GENERATED,
            magic=magic,
            module_id=module.ID,
            command_id=command.ID,
            args=kwargs
        )
    ]
)
packet = IrisCommonPacket(payloads=payloads)
packet_bytes = packet.encode()
packet_bytes_dump = scp.hexdump(packet_bytes, dump=True)
print(packet_bytes_dump)


ReportStatus   :   True
DataPathway.WIRED Magic.WATCHDOG_COMMAND WatchDogInterface_RequestStatusReport OrderedDict([('confirm', 'CONFIRM_REQUEST')])
0000  00 07 00 EE EE FF 00 C0 F6 10 57                 ..........W


In [17]:
tlm_export = """
2022-11-11T14:39:45.446Z	721449654.630	FF000002
2022-11-11T14:40:11.665Z	721449680.849	D5000818000000000038000000000067004140884000C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:40:11.706Z	721449680.890	FF000000
2022-11-11T14:40:37.805Z	721449706.989	FF000000
2022-11-11T14:41:03.977Z	721449733.161	FF000000
2022-11-11T14:41:30.193Z	721449759.377	D50008180001000000380000000000670021408F4000C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:41:30.234Z	721449759.418	FF000000
2022-11-11T14:41:56.324Z	721449785.508	FF000000
2022-11-11T14:42:22.493Z	721449811.677	FF000000
2022-11-11T14:42:48.700Z	721449837.884	D50008180002000000380000000000670041400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:42:48.741Z	721449837.925	FF000000
2022-11-11T14:43:14.820Z	721449864.004	FF000000
2022-11-11T14:43:41.000Z	721449890.184	FF000000
2022-11-11T14:44:07.223Z	721449916.407	D50008180003000000380000000000670221408F4000C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:44:07.265Z	721449916.449	FF000002
2022-11-11T14:44:33.360Z	721449942.544	FF000000
2022-11-11T14:44:59.557Z	721449968.741	FF000000
2022-11-11T14:45:25.784Z	721449994.968	D50008180004000000380000000000670041408F4000C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:45:25.825Z	721449995.009	FF000000
2022-11-11T14:45:51.929Z	721450021.113	FF000000
2022-11-11T14:46:18.105Z	721450047.289	FF000000
2022-11-11T14:46:31.240Z	721450060.424	D50008180005000000380000000000670021408F4000C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:46:44.324Z	721450073.508	D50008180006000000380000000000670041408F4000C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:46:44.366Z	721450073.550	FF000000
2022-11-11T14:47:10.459Z	721450099.643	FF000000
2022-11-11T14:47:36.634Z	721450125.818	FF000000
2022-11-11T14:48:02.841Z	721450152.025	D50008180007000000380000000000670041400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:48:02.882Z	721450152.066	FF000000
2022-11-11T14:48:28.958Z	721450178.142	FF000000
2022-11-11T14:48:55.118Z	721450204.302	FF000000
2022-11-11T14:49:21.327Z	721450230.511	D50008180008000000380000000000670041400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:49:21.368Z	721450230.552	FF000000
2022-11-11T14:49:47.444Z	721450256.628	FF000000
2022-11-11T14:50:13.617Z	721450282.801	FF000000
2022-11-11T14:50:39.839Z	721450309.023	D50008180009000000380000000000670021400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:50:39.881Z	721450309.065	FF000000
2022-11-11T14:51:05.987Z	721450335.171	FF000002
2022-11-11T14:51:32.175Z	721450361.359	FF000000
2022-11-11T14:51:58.426Z	721450387.610	D5000818000A000000380000000000670041400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:51:58.467Z	721450387.651	FF000000
2022-11-11T14:52:24.583Z	721450413.767	FF00000F
2022-11-11T14:52:50.789Z	721450439.973	FF000000
2022-11-11T14:53:17.035Z	721450466.219	D5000818000B000000380000000000670041800F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:53:17.077Z	721450466.261	FF000000
2022-11-11T14:53:43.195Z	721450492.379	FF000000
2022-11-11T14:54:09.380Z	721450518.564	FF000000
2022-11-11T14:54:09.464Z	721450518.648	D5000818000C000000380000000000670021400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:54:35.589Z	721450544.773	D5000818000D000000380000000000670041808F4000C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:54:35.630Z	721450544.814	FF000000
2022-11-11T14:55:01.730Z	721450570.914	FF000000
2022-11-11T14:55:27.933Z	721450597.117	FF000000
2022-11-11T14:55:54.174Z	721450623.358	D5000818000E000000380000000000E70041400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:55:54.215Z	721450623.399	FF000000
2022-11-11T14:56:20.334Z	721450649.518	FF000001
2022-11-11T14:56:46.529Z	721450675.713	FF000000
2022-11-11T14:57:12.767Z	721450701.951	D5000818000F000000380000000000670721400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:57:12.808Z	721450701.992	FF000007
2022-11-11T14:57:38.913Z	721450728.097	FF000000
2022-11-11T14:58:05.092Z	721450754.276	FF000000
2022-11-11T14:58:31.320Z	721450780.504	D50008180010000000380000000000E70141408F4000C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:58:31.362Z	721450780.546	FF000001
2022-11-11T14:58:57.457Z	721450806.641	FF000000
2022-11-11T14:59:23.659Z	721450832.843	FF000000
2022-11-11T14:59:49.902Z	721450859.086	D50008180011000000380000000000670141400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T14:59:49.943Z	721450859.127	FF000001
2022-11-11T15:00:16.059Z	721450885.243	FF000000
2022-11-11T15:00:42.270Z	721450911.454	FF000000
2022-11-11T15:01:08.520Z	721450937.704	D50008180012000000380000000000670021400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:01:08.561Z	721450937.745	FF000000
2022-11-11T15:01:34.681Z	721450963.865	FF000001
2022-11-11T15:01:47.825Z	721450977.009	D50008180013000000380000000000670141400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:02:00.881Z	721450990.065	FF000000
2022-11-11T15:02:27.130Z	721451016.314	D50008180014000000380000000000670141408F4000C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:02:27.171Z	721451016.355	FF000001
2022-11-11T15:02:53.286Z	721451042.470	FF000000
2022-11-11T15:03:19.478Z	721451068.662	FF000000
2022-11-11T15:03:45.717Z	721451094.901	D50008180015000000380000000000670021408F4000C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:03:45.759Z	721451094.943	FF000000
2022-11-11T15:04:11.885Z	721451121.069	FF000000
2022-11-11T15:04:38.088Z	721451147.272	FF000000
2022-11-11T15:05:04.318Z	721451173.502	D50008180016000000380000000000670041400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:05:04.359Z	721451173.543	FF000000
2022-11-11T15:05:30.463Z	721451199.647	FF000000
2022-11-11T15:05:56.646Z	721451225.830	FF000000
2022-11-11T15:06:22.879Z	721451252.063	D50008180017000000380000000000670041800F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:06:22.920Z	721451252.104	FF000000
2022-11-11T15:06:49.032Z	721451278.216	FF000000
2022-11-11T15:07:15.237Z	721451304.421	FF00000E
2022-11-11T15:07:41.480Z	721451330.664	D50008180018000000380000000000670641808F4000C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:07:41.521Z	721451330.705	FF000006
2022-11-11T15:08:07.625Z	721451356.809	FF000000
2022-11-11T15:08:33.807Z	721451382.991	FF000004
2022-11-11T15:09:00.028Z	721451409.212	D50008180019000000380000000000E70041800F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:09:00.069Z	721451409.253	FF000000
2022-11-11T15:09:26.159Z	721451435.343	FF000000
2022-11-11T15:09:26.242Z	721451435.426	D5000818001A000000380000000000671341400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:09:52.334Z	721451461.518	FF000000
2022-11-11T15:10:18.533Z	721451487.717	D5000818001B000000380000000000670041400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:10:18.574Z	721451487.758	FF000000
2022-11-11T15:10:44.664Z	721451513.848	FF000000
2022-11-11T15:11:10.836Z	721451540.020	FF000015
2022-11-11T15:11:37.046Z	721451566.230	D5000818001C000000380000000000E70041400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:11:37.088Z	721451566.272	FF000000
2022-11-11T15:12:03.182Z	721451592.366	FF000007
2022-11-11T15:12:29.362Z	721451618.546	FF000000
2022-11-11T15:12:55.594Z	721451644.778	D5000818001D000000380000000000E71821400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:12:55.636Z	721451644.820	FF000018
2022-11-11T15:13:21.736Z	721451670.920	FF000000
2022-11-11T15:13:47.933Z	721451697.117	FF000000
2022-11-11T15:14:14.179Z	721451723.363	D5000818001E000000380000000000670841400F4100C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:14:14.221Z	721451723.405	FF000008
2022-11-11T15:14:40.330Z	721451749.514	FF000000
2022-11-11T15:15:06.523Z	721451775.707	FF000000
2022-11-11T15:15:29.829Z	721451799.013	D5000818001F000000380000000000670041408F4000C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:15:29.862Z	721451799.046	0AF600
2022-11-11T15:15:29.949Z	721451799.133	D50008180020000000380000000000670041408F4000C4F4013421FD0C560E180D1027000000000000000000000000
2022-11-11T15:15:32.727Z	721451801.911	FF000000

"""

In [18]:
# All telem received (from YAMCS):
packet_strs = [line.split('\t') for line in tlm_export.split('\n') if len(line) > 2]
packet_strs = [(line[0], line[-1]) for line in packet_strs]

In [19]:
packet_bytes = [(t, bytes.fromhex(p)) for t,p in packet_strs]
packets = [(t, parse_packet(pb)) for t, pb in packet_bytes]
for t, packet in packets:
    cprint(f"\n\nReceived at SCET {t}:", 'green')
    print(packet)



Received at SCET 2022-11-11T14:39:45.446Z:
[HEAT]: OFF 		 [BATT]: Voltage: BAD, Charge:   29mAh =   0.8%, Temp:  74.3°K]


Received at SCET 2022-11-11T14:40:11.665Z:
#0 [RS_KEEP_ALIVE + NOT_DEPLOYED] 	CHARGER: OFF 	BATTERIES: DISCONNECTED 	Herc-UART0: OFF 	Lander-UART1: INITIALIZED_&_ACTIVE
1V2: BAD 	1V8: BAD 	2V5: 0.0 	2V8: 0.0 	3V3: BAD 	5V0: BAD 	24V: 0.21
VBm: 1.77V 	VL: 25.55V 	Vcc28: 27.80V 	VSA: 0.99V 	ISA: 7.4mA
HEATER [268K |274K |273K] 	 is NOT_HEATING, Control: ENABLED 	@ 0.000W / 0.796W 	Period: 1.2ms
Tbatt: 428.1K ± 0.0K 	Tchrg: 0.0K ± 0.0K 	Tblimp: 0.0K
BATTERY MONITOR: 0.00V 	0.0mA 	0mAh 	
       2V5    2V8    24V    VBm      VL   Vcc28    VSA    ISA       Tbatt     Tchrg
ADC      0      0     32    264    3296    3136    128      2           1         0
val  0.00V  0.00V  0.21V  1.77V  25.55V  27.80V  0.99V  7.4mA  428.1±0.0K  0.0±0.0K
              HEAT  DEP  BSTAT  BE  LB  LS  LR  CE  REGE  VSAE  3V3_EN  H_ON  R_ON  F_ON  M_ON  H_RST  H_N_PRST  F_N_RST  R_N_RST  CAM

In [16]:
len(packets)

82